<h1> Capstone Project - The Battle of Neighborhoods (Week 2)</h1> 

<h2>Business Problem section</h2>

<h3>Background</h3>

<p>According to Bloomberg News, the London Housing Market is in a rut. It is now facing a number of different headwinds, including the prospect of higher taxes and a warning from the Bank of England that U.K. home values could fall as much as 30 percent in the event of a disorderly exit from the European Union. More specifically, four overlooked cracks suggest that the London market may be in worse shape than many realize: hidden price falls, record-low sales, homebuilder exodus and tax hikes addressing overseas buyers of homes in England and Wales.</p>

<h3>Business Problem</h3>

<p>In this scenario, it is urgent to adopt machine learning tools in order to assist homebuyers clientele in London to make wise and effective decisions. As a result, the business problem we are currently posing is: how could we provide support to homebuyers clientele in to purchase a suitable real estate in London in this uncertain economic and financial scenario?

To solve this business problem, we are going to cluster London neighborhoods in order to recommend venues and the current average price of real estate where homebuyers can make a real estate investment. We will recommend profitable venues according to amenities and essential facilities surrounding such venues i.e. elementary schools, high schools, hospitals & grocery stores.</p>

<h2>Data section</h2>

<p>Data on London properties and the relative price paid data were extracted from the HM Land Registry (http://landregistry.data.gov.uk/). The following fields comprise the address data included in Price Paid Data: Postcode; PAON Primary Addressable Object Name. Typically the house number or name; SAON Secondary Addressable Object Name. If there is a sub-building, for example, the building is divided into flats, there will be a SAON; Street; Locality; Town/City; District; County.

To explore and target recommended locations across different venues according to the presence of amenities and essential facilities, we will access data through FourSquare API interface and arrange them as a dataframe for visualization. By merging data on London properties and the relative price paid data from the HM Land Registry and data on amenities and essential facilities surrounding such properties from FourSquare API interface, we will be able to recommend profitable real estate investments.</p>

<h2> Methodology section</h2>

The Methodology section will describe the main components of our analysis and predication system. The Methodology section comprises four stages:

1. Collect Inspection Data
2. Explore and Understand Data
3. Data preparation and preprocessing 
4. Modeling

<h3>1. Collect Inspection Data</h3>
<p>After importing the necessary libraries, we download the data from the HM Land Registry website as follows:</p>

In [1]:
import os # Operating System
import numpy as np
import pandas as pd
import datetime as dt # Datetime
import json # library to handle JSON files

!conda install -c conda-forge geopy --yes
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

!conda install -c conda-forge folium=0.5.0 --yes
import folium #import folium # map rendering library

print('Libraries imported.')

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.8.2
  latest version: 4.8.3

Please update conda by running

    $ conda update -n base -c defaults conda



## Package Plan ##

  environment location: /home/sudhanva/anaconda3

  added / updated specs:
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    certifi-2019.11.28         |   py37hc8dfbb8_1         149 KB  conda-forge
    conda-4.8.3                |   py37hc8dfbb8_1         3.0 MB  conda-forge
    openssl-1.1.1e             |       h516909a_0         2.1 MB  conda-forge
    python_abi-3.7             |          1_cp37m           4 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         5.3 MB

The following NEW packages will be INSTALLED:

  python_abi         conda-forge/linux-64::python

In [2]:
#Read the data for examination (Source: http://landregistry.data.gov.uk/)
df_ppd = pd.read_csv("http://prod2.publicdata.landregistry.gov.uk.s3-website-eu-west-1.amazonaws.com/pp-2018.csv")

<p>Before using data, we will have to explore and understand it.</p>
<h3>2. Explore and Understand Data</h3>
<p>We read the dataset that we collected from the HM Land Registry website into a pandas' data frame and display the first five rows of it as follows:</p>

In [3]:
df_ppd.head(5)

,{666758D7-43A9-3363-E053-6B04A8C0D74E},405000,2018-01-25 00:00,WR15 8LH,D,N,F,RAMBLERS WAY,Unnamed: 8,Unnamed: 9,BORASTON,TENBURY WELLS,SHROPSHIRE,SHROPSHIRE.1,A,A.1
0,{666758D7-43AA-3363-E053-6B04A8C0D74E},315000,2018-01-23 00:00,SY7 8QA,D,N,F,MONT CENISE,NaN,NaN,CLUN,CRAVEN ARMS,SHROPSHIRE,SHROPSHIRE,A,A
1,{666758D7-43AD-3363-E053-6B04A8C0D74E},165000,2018-01-19 00:00,SY1 2BF,T,Y,F,42,NaN,PENSON WAY,NaN,SHREWSBURY,SHROPSHIRE,SHROPSHIRE,A,A
2,{666758D7-43B0-3363-E053-6B04A8C0D74E},370000,2018-01-22 00:00,SY8 4DF,D,N,F,WILLOW HEY,NaN,NaN,ASHFORD CARBONEL,LUDLOW,SHROPSHIRE,SHROPSHIRE,A,A
3,{666758D7-43B3-3363-E053-6B04A8C0D74E},320000,2018-01-19 00:00,TF10 7ET,D,N,F,3,NaN,PRINCESS GARDENS,NaN,NEWPORT,WREKIN,WREKIN,A,A
4,{666758D7-43B4-3363-E053-6B04A8C0D74E},180000,2018-01-31 00:00,SY3 0NQ,S,N,F,79,NaN,LYTHWOOD ROAD,BAYSTON HILL,SHREWSBURY,SHROPSHIRE,SHROPSHIRE,A,A


In [4]:
df_ppd.shape

(1028554, 16)

<p>Our dataset consists of over 700000 rows and 16 columns. We will now prepare and preprocess data accordingly.</p>
<h3>3. Data preparation and preprocessing</h3>
<p>At this stage, we prepare our dataset for the modeling process, opting for the most suitable machine learning algorithm for our scope. Accordingly, we perform the following steps:
<ul>
    <li>Rename the column names</li>
    <li>Format the date column</li>
    <li>Sort data by date of sale</li>
    <li>Select data only for the city of London</li>
    <li>Make a list of street names in London</li>
    <li>Calculate the street-wise average price of the property</li>
<li>Read the street-wise coordinates into a data frame, eliminating recurring word London from individual names</li>
    <li>Join the data to find the coordinates of locations which fit into client's budget</li>
    <li>Plot recommended locations on London map along with current market prices</li>

In [5]:
# Assign meaningful column names
df_ppd.columns = ['TUID', 'Price', 'Date_Transfer', 'Postcode', 'Prop_Type', 'Old_New', 'Duration', 'PAON', \
                  'SAON', 'Street', 'Locality', 'Town_City', 'District', 'County', 'PPD_Cat_Type', 'Record_Status']

In [6]:
# Format the date column
df_ppd['Date_Transfer'] = df_ppd['Date_Transfer'].apply(pd.to_datetime)

# Delete all obsolete transactions which were done before 2016
df_ppd.drop(df_ppd[df_ppd.Date_Transfer.dt.year < 2016].index, inplace=True)

# Sort by Date of Sale
df_ppd.sort_values(by=['Date_Transfer'],ascending=[False],inplace=True)

In [7]:
df_ppd_london = df_ppd.query("Town_City == 'LONDON'")

# Make a list of street names in LONDON
streets = df_ppd_london['Street'].unique().tolist()

In [8]:
df_grp_price = df_ppd_london.groupby(['Street'])['Price'].mean().reset_index()

# Give meaningful names to the columns
df_grp_price.columns = ['Street', 'Avg_Price']

In [9]:
#Input your Budget's Upper Limit and Lower Limit - Find the locations df_grp_price which fits your budget
df_affordable = df_grp_price.query("(Avg_Price >= 2200000) & (Avg_Price <= 2500000)")

In [10]:
# Display the dataframe
df_affordable

,Street,Avg_Price
196,ALBION SQUARE,2.450000e+06
390,ANHALT ROAD,2.435000e+06
405,ANSDELL TERRACE,2.250000e+06
422,APPLEGARTH ROAD,2.400000e+06
855,BARONSMEAD ROAD,2.375000e+06
981,BEAUCLERC ROAD,2.480000e+06
1102,BELVEDERE DRIVE,2.340000e+06
1215,BICKENHALL STREET,2.208500e+06
1253,BIRCHLANDS AVENUE,2.217000e+06
1553,BRAMPTON GROVE,2.456875e+06


In [11]:
import pandas as pd
import numpy as np
import datetime as DT
import hmac
from geopy.geocoders import Nominatim
from geopy.distance import vincenty
# import k-means from clustering stage
from sklearn.cluster import KMeans

In [12]:
for index, item in df_affordable.iterrows():
    print(f"index: {index}")
    print(f"item: {item}")
    print(f"item.Street only: {item.Street}")

index: 196
item: Street       ALBION SQUARE
Avg_Price         2.45e+06
Name: 196, dtype: object
item.Street only: ALBION SQUARE
index: 390
item: Street       ANHALT ROAD
Avg_Price      2.435e+06
Name: 390, dtype: object
item.Street only: ANHALT ROAD
index: 405
item: Street       ANSDELL TERRACE
Avg_Price           2.25e+06
Name: 405, dtype: object
item.Street only: ANSDELL TERRACE
index: 422
item: Street       APPLEGARTH ROAD
Avg_Price            2.4e+06
Name: 422, dtype: object
item.Street only: APPLEGARTH ROAD
index: 855
item: Street       BARONSMEAD ROAD
Avg_Price          2.375e+06
Name: 855, dtype: object
item.Street only: BARONSMEAD ROAD
index: 981
item: Street       BEAUCLERC ROAD
Avg_Price          2.48e+06
Name: 981, dtype: object
item.Street only: BEAUCLERC ROAD
index: 1102
item: Street       BELVEDERE DRIVE
Avg_Price           2.34e+06
Name: 1102, dtype: object
item.Street only: BELVEDERE DRIVE
index: 1215
item: Street       BICKENHALL STREET
Avg_Price           2.2085e+06
N

In [13]:
geolocator = Nominatim()

/home/sudhanva/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Using Nominatim with the default "geopy/1.21.0" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  """Entry point for launching an IPython kernel.


In [15]:
df_affordable['city_coord'] = df_affordable['Street'].apply(geolocator.geocode).apply(lambda x: (x.latitude, x.longitude))

/home/sudhanva/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [16]:
df_affordable

,Street,Avg_Price,city_coord
196,ALBION SQUARE,2.450000e+06,"(-41.27375755, 173.28939323910353)"
390,ANHALT ROAD,2.435000e+06,"(51.4803164, -0.1668011)"
405,ANSDELL TERRACE,2.250000e+06,"(51.4998899, -0.1891027)"
422,APPLEGARTH ROAD,2.400000e+06,"(53.7486539, -0.3266704)"
855,BARONSMEAD ROAD,2.375000e+06,"(51.4773147, -0.239457)"
981,BEAUCLERC ROAD,2.480000e+06,"(30.2114523, -81.6179807)"
1102,BELVEDERE DRIVE,2.340000e+06,"(40.482358, -86.106465)"
1215,BICKENHALL STREET,2.208500e+06,"(51.5212014, -0.1589082)"
1253,BIRCHLANDS AVENUE,2.217000e+06,"(51.4483941, -0.1604676)"
1553,BRAMPTON GROVE,2.456875e+06,"(51.5899607, -0.3185249)"


In [17]:
df_affordable[['Latitude', 'Longitude']] = df_affordable['city_coord'].apply(pd.Series)

/home/sudhanva/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:3391: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[k1] = value[k2]


In [18]:
df_affordable

,Street,Avg_Price,city_coord,Latitude,Longitude
196,ALBION SQUARE,2.450000e+06,"(-41.27375755, 173.28939323910353)",-41.273758,173.289393
390,ANHALT ROAD,2.435000e+06,"(51.4803164, -0.1668011)",51.480316,-0.166801
405,ANSDELL TERRACE,2.250000e+06,"(51.4998899, -0.1891027)",51.499890,-0.189103
422,APPLEGARTH ROAD,2.400000e+06,"(53.7486539, -0.3266704)",53.748654,-0.326670
855,BARONSMEAD ROAD,2.375000e+06,"(51.4773147, -0.239457)",51.477315,-0.239457
981,BEAUCLERC ROAD,2.480000e+06,"(30.2114523, -81.6179807)",30.211452,-81.617981
1102,BELVEDERE DRIVE,2.340000e+06,"(40.482358, -86.106465)",40.482358,-86.106465
1215,BICKENHALL STREET,2.208500e+06,"(51.5212014, -0.1589082)",51.521201,-0.158908
1253,BIRCHLANDS AVENUE,2.217000e+06,"(51.4483941, -0.1604676)",51.448394,-0.160468
1553,BRAMPTON GROVE,2.456875e+06,"(51.5899607, -0.3185249)",51.589961,-0.318525


In [19]:
df = df_affordable.drop(columns=['city_coord'])

In [20]:
df

,Street,Avg_Price,Latitude,Longitude
196,ALBION SQUARE,2.450000e+06,-41.273758,173.289393
390,ANHALT ROAD,2.435000e+06,51.480316,-0.166801
405,ANSDELL TERRACE,2.250000e+06,51.499890,-0.189103
422,APPLEGARTH ROAD,2.400000e+06,53.748654,-0.326670
855,BARONSMEAD ROAD,2.375000e+06,51.477315,-0.239457
981,BEAUCLERC ROAD,2.480000e+06,30.211452,-81.617981
1102,BELVEDERE DRIVE,2.340000e+06,40.482358,-86.106465
1215,BICKENHALL STREET,2.208500e+06,51.521201,-0.158908
1253,BIRCHLANDS AVENUE,2.217000e+06,51.448394,-0.160468
1553,BRAMPTON GROVE,2.456875e+06,51.589961,-0.318525


In [21]:
address = 'London, UK'

geolocator = Nominatim()
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of London City are {}, {}.'.format(latitude, longitude))

/home/sudhanva/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: DeprecationWarning: Using Nominatim with the default "geopy/1.21.0" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  This is separate from the ipykernel package so we can avoid doing imports until


The geograpical coordinate of London City are 51.5073219, -0.1276474.


In [22]:
# create map of London using latitude and longitude values
map_london = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, price, street in zip(df['Latitude'], df['Longitude'], df['Avg_Price'], df['Street']):
    label = '{}, {}'.format(street, price)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_london)  
    
map_london

In [37]:
#Define Foursquare Credentials and Version

CLIENT_ID = 'S3L1ZBESMLGBWQUMCMQURIZ0ODPO1AZGCI0LTISO25NSAY5U' # Foursquare ID
CLIENT_SECRET = 'JWCMFFKVARR0R1RLTLSQW5YK3TIZOX4CDI1KTZE43UI4WNFU' # Foursquare Secret
VERSION = '20181206' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)


Your credentails:
CLIENT_ID: S3L1ZBESMLGBWQUMCMQURIZ0ODPO1AZGCI0LTISO25NSAY5U
CLIENT_SECRET:JWCMFFKVARR0R1RLTLSQW5YK3TIZOX4CDI1KTZE43UI4WNFU


<p>We can now proceed to the Modeling phase. We will analyze neighborhoods to recommend real estates where home buyers can make a real estate investment. We will then recommend profitable venues according to amenities and essential facilities surrounding such venues i.e. elementary schools, high schools, hospitals & grocery stores.</p>
<h3>4. Modeling</h3>
<p>After exploring the dataset and gaining insights into it, we are ready to use the clustering methodology to analyze real estates. We will use the k-means clustering technique as it is fast and efficient in terms of computational cost, is highly flexible to account for mutations in real estate market in London and is accurate.</p>

In [38]:
def getNearbyVenues(names, latitudes, longitudes, radius=500, LIMIT=100):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]["groups"][0]["items"]
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Street', 
                  'Street Latitude', 
                  'Street Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [39]:
# Run the above function on each location and create a new dataframe called location_venues and display it.
location_venues = getNearbyVenues(names=df['Street'],
                                   latitudes=df['Latitude'],
                                   longitudes=df['Longitude']
                                  )

ALBION SQUARE
ANHALT ROAD
ANSDELL TERRACE
APPLEGARTH ROAD
BARONSMEAD ROAD
BEAUCLERC ROAD
BELVEDERE DRIVE
BICKENHALL STREET
BIRCHLANDS AVENUE
BRAMPTON GROVE
BRIARDALE GARDENS
BROOKWAY
BURBAGE ROAD
BURY WALK
CALLCOTT STREET
CAMPDEN HILL ROAD
CAMPION ROAD
CANNING PLACE
CARLISLE ROAD
CARLTON GARDENS
CARLYLE COURT
CHALCOT SQUARE
CHARLES LANE
CHELSEA CRESCENT
CHESTER CLOSE NORTH
CHEYNE COURT
CHEYNE ROW
CHISWICK MALL
CITY ROAD
CLARENDON STREET
CLONCURRY STREET
COLBECK MEWS
COLLEGE CRESCENT
CORNWALL TERRACE MEWS
COURT LANE GARDENS
CRESCENT GROVE
DALEBURY ROAD
DEWHURST ROAD
DORIA ROAD
DOWNSHIRE HILL
DUCHESS WALK
ECCLESTON SQUARE MEWS
EGBERT STREET
EGERTON PLACE
ELM PARK ROAD
FLORAL STREET
FRANK DIXON WAY
FULTON MEWS
GERARD ROAD
GERRARD ROAD
GIRDLERS ROAD
GLOUCESTER CRESCENT
GORDON PLACE
GRAFTON SQUARE
GRAHAM TERRACE
HARMAN DRIVE
HARRIS STREET
HAVANNAH STREET
HAZLEWELL ROAD
HEREFORD MEWS
HERONDALE AVENUE
HIGHGATE HIGH STREET
HIGHWOOD HILL
HILLGATE PLACE
HOLLYCROFT AVENUE
HOLLYWOOD MEWS
HONEYWELL

In [40]:
location_venues

,Street,Street Latitude,Street Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,ALBION SQUARE,-41.273758,173.289393,The Free House,-41.273340,173.287364,Bar
1,ALBION SQUARE,-41.273758,173.289393,The Indian Cafe,-41.273308,173.286530,Indian Restaurant
2,ALBION SQUARE,-41.273758,173.289393,Queen's Gardens,-41.273671,173.291383,Park
3,ALBION SQUARE,-41.273758,173.289393,The Bridge Street Collective,-41.272520,173.285517,Café
4,ALBION SQUARE,-41.273758,173.289393,Urban,-41.274355,173.286317,New American Restaurant
5,ALBION SQUARE,-41.273758,173.289393,Fish Stop,-41.276010,173.289592,Fish & Chips Shop
6,ALBION SQUARE,-41.273758,173.289393,Deville Cafe,-41.271941,173.285535,Beer Garden
7,ALBION SQUARE,-41.273758,173.289393,The Kitchen,-41.272360,173.285500,Café
8,ALBION SQUARE,-41.273758,173.289393,Burger Culture,-41.274750,173.284030,Burger Joint
9,ALBION SQUARE,-41.273758,173.289393,The Vic Mac's Brew Bar,-41.274757,173.283914,Pub


In [41]:

location_venues.groupby('Street').count()

,Street Latitude,Street Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Street,,,,,,
ALBION SQUARE,30,30,30,30,30,30
ANHALT ROAD,16,16,16,16,16,16
ANSDELL TERRACE,47,47,47,47,47,47
APPLEGARTH ROAD,4,4,4,4,4,4
BARONSMEAD ROAD,15,15,15,15,15,15
BEAUCLERC ROAD,4,4,4,4,4,4
BELVEDERE DRIVE,9,9,9,9,9,9
BICKENHALL STREET,87,87,87,87,87,87
BIRCHLANDS AVENUE,10,10,10,10,10,10


In [42]:
# get the List of Unique Categories
print('There are {} uniques categories.'.format(len(location_venues['Venue Category'].unique())))

There are 333 uniques categories.


In [43]:
location_venues.shape

(4429, 7)

In [44]:
# one hot encoding
venues_onehot = pd.get_dummies(location_venues[['Venue Category']], prefix="", prefix_sep="")

# add street column back to dataframe
venues_onehot['Street'] = location_venues['Street'] 

# move street column to the first column
fixed_columns = [venues_onehot.columns[-1]] + list(venues_onehot.columns[:-1])

#fixed_columns
venues_onehot = venues_onehot[fixed_columns]

venues_onehot.head()

,Street,Accessories Store,Adult Boutique,Afghan Restaurant,African Restaurant,American Restaurant,Antique Shop,Arcade,Argentinian Restaurant,Art Gallery,...,Vietnamese Restaurant,Warehouse Store,Waterfront,Weight Loss Center,Windmill,Wine Bar,Wine Shop,Women's Store,Yoga Studio,Zoo
0,ALBION SQUARE,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,ALBION SQUARE,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,ALBION SQUARE,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,ALBION SQUARE,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,ALBION SQUARE,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [45]:
london_grouped = venues_onehot.groupby('Street').mean().reset_index()
london_grouped

,Street,Accessories Store,Adult Boutique,Afghan Restaurant,African Restaurant,American Restaurant,Antique Shop,Arcade,Argentinian Restaurant,Art Gallery,...,Vietnamese Restaurant,Warehouse Store,Waterfront,Weight Loss Center,Windmill,Wine Bar,Wine Shop,Women's Store,Yoga Studio,Zoo
0,ALBION SQUARE,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.033333,...,0.000000,0.00,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0
1,ANHALT ROAD,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.000000,...,0.000000,0.00,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0
2,ANSDELL TERRACE,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.021277,...,0.000000,0.00,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0
3,APPLEGARTH ROAD,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.000000,...,0.000000,0.00,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0
4,BARONSMEAD ROAD,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.000000,...,0.000000,0.00,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0
5,BEAUCLERC ROAD,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.000000,...,0.000000,0.00,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0
6,BELVEDERE DRIVE,0.000000,0.0,0.0,0.0,0.111111,0.000000,0.0,0.000000,0.000000,...,0.000000,0.00,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0
7,BICKENHALL STREET,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.000000,...,0.000000,0.00,0.000000,0.0,0.0,0.011494,0.0,0.000000,0.011494,0.0
8,BIRCHLANDS AVENUE,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.000000,...,0.000000,0.00,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0
9,BRAMPTON GROVE,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.000000,...,0.000000,0.00,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0


In [46]:
london_grouped.shape

(150, 334)

In [47]:
# What are the top 5 venues/facilities nearby profitable real estate investments?#

num_top_venues = 5

for hood in london_grouped['Street']:
    print("----"+hood+"----")
    temp = london_grouped[london_grouped['Street'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----ALBION SQUARE----
               venue  freq
0               Café  0.20
1                Bar  0.07
2         Restaurant  0.07
3  Indian Restaurant  0.07
4                Pub  0.07


----ANHALT ROAD----
                 venue  freq
0                  Pub  0.25
1        Grocery Store  0.12
2  Japanese Restaurant  0.06
3         Cocktail Bar  0.06
4               Garden  0.06


----ANSDELL TERRACE----
               venue  freq
0     Clothing Store  0.09
1         Restaurant  0.09
2          Juice Bar  0.06
3              Hotel  0.06
4  French Restaurant  0.04


----APPLEGARTH ROAD----
                   venue  freq
0                    Pub  0.25
1                 Casino  0.25
2                    Bar  0.25
3              Nightclub  0.25
4  Portuguese Restaurant  0.00


----BARONSMEAD ROAD----
                 venue  freq
0  Indie Movie Theater  0.07
1      Thai Restaurant  0.07
2           Restaurant  0.07
3     Community Center  0.07
4          Coffee Shop  0.07


----BEAUCLERC ROAD

                        venue  freq
0           Convenience Store  0.25
1           Indian Restaurant  0.25
2           Fish & Chips Shop  0.25
3  Construction & Landscaping  0.25
4         Japanese Restaurant  0.00


----GERRARD ROAD----
                        venue  freq
0  Construction & Landscaping  0.33
1         Fried Chicken Joint  0.33
2                         Pub  0.33
3          Italian Restaurant  0.00
4         Japanese Restaurant  0.00


----GIRDLERS ROAD----
                venue  freq
0                 Pub  0.14
1         Pizza Place  0.06
2  Italian Restaurant  0.06
3                Café  0.06
4           Gastropub  0.06


----GORDON PLACE----
                venue  freq
0  Seafood Restaurant   1.0
1        Perfume Shop   0.0
2    Pedestrian Plaza   0.0
3         Pastry Shop   0.0
4                Park   0.0


----GRAFTON SQUARE----
          venue  freq
0           Pub  0.09
1    Restaurant  0.06
2          Café  0.04
3  Burger Joint  0.04
4           Bar  0.04


---

                   venue  freq
0                    Pub   0.4
1                   Pool   0.2
2   Fast Food Restaurant   0.2
3                 Bakery   0.2
4  Portuguese Restaurant   0.0


----OAKWOOD COURT----
               venue  freq
0               Food  0.25
1         Smoke Shop  0.25
2        Golf Course  0.25
3        Pizza Place  0.25
4  Accessories Store  0.00


----OBSERVATORY GARDENS----
              venue  freq
0              Café  0.06
1    Clothing Store  0.05
2               Pub  0.05
3         Juice Bar  0.04
4  Greek Restaurant  0.03


----OLD COURT PLACE----
            venue  freq
0           Hotel  0.08
1          Garden  0.06
2  Clothing Store  0.06
3       Juice Bar  0.05
4      Restaurant  0.05


----ONSLOW MEWS WEST----
                venue  freq
0               Hotel  0.11
1              Bakery  0.06
2  Italian Restaurant  0.05
3      Sandwich Place  0.04
4         Coffee Shop  0.04


----PALACE PLACE----
                   venue  freq
0      Electronics Stor

                  venue  freq
0           Supermarket   0.6
1  Fast Food Restaurant   0.2
2           Pizza Place   0.2
3     Accessories Store   0.0
4  Outdoor Supply Store   0.0


----SUTHERLAND PLACE----
               venue  freq
0               Park   0.5
1              Hotel   0.5
2  Accessories Store   0.0
3  Outdoor Sculpture   0.0
4        Pastry Shop   0.0


----THAMES BANK----
                  venue  freq
0  Gym / Fitness Center  0.25
1         Grocery Store  0.25
2          Burger Joint  0.25
3           Pizza Place  0.25
4     Accessories Store  0.00


----THE HEXAGON----
                venue  freq
0      Clothing Store  0.12
1                 Pub  0.09
2         Coffee Shop  0.07
3    Asian Restaurant  0.07
4  Italian Restaurant  0.07


----TREDEGAR SQUARE----
           venue  freq
0            Pub  0.16
1       Bus Stop  0.10
2    Pizza Place  0.10
3    Coffee Shop  0.06
4  Grocery Store  0.06


----TRINITY STREET----
                    venue  freq
0          Clothin

In [48]:
# Define a function to return the most common venues/facilities nearby real estate investments#

def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [49]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Street']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

In [50]:
# create a new dataframe
venues_sorted = pd.DataFrame(columns=columns)
venues_sorted['Street'] = london_grouped['Street']

for ind in np.arange(london_grouped.shape[0]):
    venues_sorted.iloc[ind, 1:] = return_most_common_venues(london_grouped.iloc[ind, :], num_top_venues)

In [51]:
venues_sorted.head()

,Street,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,ALBION SQUARE,Café,Bar,Indian Restaurant,Restaurant,Pub,Coffee Shop,Museum,Burger Joint,Supermarket,Seafood Restaurant
1,ANHALT ROAD,Pub,Grocery Store,Japanese Restaurant,English Restaurant,Gym / Fitness Center,Diner,Garden,French Restaurant,Park,Plaza
2,ANSDELL TERRACE,Clothing Store,Restaurant,Hotel,Juice Bar,Indian Restaurant,Italian Restaurant,Bakery,French Restaurant,English Restaurant,Pub
3,APPLEGARTH ROAD,Pub,Casino,Bar,Nightclub,Fast Food Restaurant,Event Space,Exhibit,Factory,Falafel Restaurant,Farm
4,BARONSMEAD ROAD,Indie Movie Theater,Thai Restaurant,Movie Theater,Food & Drink Shop,Nature Preserve,Farmers Market,Park,Pizza Place,Pub,Coffee Shop


In [52]:
venues_sorted.shape

(150, 11)

In [53]:
london_grouped.shape

(150, 334)

In [54]:
london_grouped=df


<p>After our inspection of venues/facilities/amenities nearby the most profitable real estate investments in London, we could begin by clustering properties by venues/facilities/amenities nearby.</p>

In [55]:
#Distribute in 5 Clusters

# set number of clusters
kclusters = 5

london_grouped_clustering = london_grouped.drop('Street', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(london_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:50]

array([1, 3, 0, 3, 2, 1, 2, 0, 0, 1, 3, 3, 3, 1, 2, 2, 1, 3, 0, 1, 4, 4,
       3, 1, 1, 0, 3, 4, 1, 0, 3, 2, 3, 2, 2, 4, 3, 3, 2, 0, 1, 2, 4, 0,
       4, 0, 0, 4, 0, 0], dtype=int32)

In [56]:
#Dataframe to include Clusters

london_grouped_clustering=df
london_grouped_clustering.head()

,Street,Avg_Price,Latitude,Longitude
196,ALBION SQUARE,2450000.0,-41.273758,173.289393
390,ANHALT ROAD,2435000.0,51.480316,-0.166801
405,ANSDELL TERRACE,2250000.0,51.499890,-0.189103
422,APPLEGARTH ROAD,2400000.0,53.748654,-0.326670
855,BARONSMEAD ROAD,2375000.0,51.477315,-0.239457


In [57]:

london_grouped_clustering.shape

(162, 4)

In [58]:

df.shape

(162, 4)

In [59]:
london_grouped_clustering.dtypes

Street        object
Avg_Price    float64
Latitude     float64
Longitude    float64
dtype: object

In [60]:
df.dtypes

Street        object
Avg_Price    float64
Latitude     float64
Longitude    float64
dtype: object

In [61]:
# add clustering labels
london_grouped_clustering['Cluster Labels'] = kmeans.labels_

# merge london_grouped with london_data to add latitude/longitude for each neighborhood
london_grouped_clustering = london_grouped_clustering.join(venues_sorted.set_index('Street'), on='Street')

london_grouped_clustering.head(30) # check the last columns!

,Street,Avg_Price,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
196,ALBION SQUARE,2.450000e+06,-41.273758,173.289393,1,Café,Bar,Indian Restaurant,Restaurant,Pub,Coffee Shop,Museum,Burger Joint,Supermarket,Seafood Restaurant
390,ANHALT ROAD,2.435000e+06,51.480316,-0.166801,3,Pub,Grocery Store,Japanese Restaurant,English Restaurant,Gym / Fitness Center,Diner,Garden,French Restaurant,Park,Plaza
405,ANSDELL TERRACE,2.250000e+06,51.499890,-0.189103,0,Clothing Store,Restaurant,Hotel,Juice Bar,Indian Restaurant,Italian Restaurant,Bakery,French Restaurant,English Restaurant,Pub
422,APPLEGARTH ROAD,2.400000e+06,53.748654,-0.326670,3,Pub,Casino,Bar,Nightclub,Fast Food Restaurant,Event Space,Exhibit,Factory,Falafel Restaurant,Farm
855,BARONSMEAD ROAD,2.375000e+06,51.477315,-0.239457,2,Indie Movie Theater,Thai Restaurant,Movie Theater,Food & Drink Shop,Nature Preserve,Farmers Market,Park,Pizza Place,Pub,Coffee Shop
981,BEAUCLERC ROAD,2.480000e+06,30.211452,-81.617981,1,Speakeasy,Spa,Pizza Place,Automotive Shop,Filipino Restaurant,Event Space,Exhibit,Factory,Falafel Restaurant,Farm
1102,BELVEDERE DRIVE,2.340000e+06,40.482358,-86.106465,2,Pizza Place,Discount Store,Outlet Store,American Restaurant,Salon / Barbershop,Italian Restaurant,Chinese Restaurant,Zoo,Farm,Event Space
1215,BICKENHALL STREET,2.208500e+06,51.521201,-0.158908,0,Hotel,Café,Coffee Shop,Italian Restaurant,Gastropub,Pizza Place,Restaurant,Chinese Restaurant,Museum,Salad Place
1253,BIRCHLANDS AVENUE,2.217000e+06,51.448394,-0.160468,0,French Restaurant,Pub,Brewery,Bakery,Coffee Shop,Lake,Train Station,Chinese Restaurant,Farmers Market,Event Space
1553,BRAMPTON GROVE,2.456875e+06,51.589961,-0.318525,1,Home Service,Zoo,Filipino Restaurant,Ethiopian Restaurant,Event Space,Exhibit,Factory,Falafel Restaurant,Farm,Farmers Market


In [62]:
# Create Map

map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(london_grouped_clustering['Latitude'], london_grouped_clustering['Longitude'], london_grouped_clustering['Street'], london_grouped_clustering['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [63]:
london_grouped_clustering.loc[london_grouped_clustering['Cluster Labels'] == 0, london_grouped_clustering.columns[[1] + list(range(5, london_grouped_clustering.shape[1]))]].head()

,Avg_Price,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
405,2250000.0,Clothing Store,Restaurant,Hotel,Juice Bar,Indian Restaurant,Italian Restaurant,Bakery,French Restaurant,English Restaurant,Pub
1215,2208500.0,Hotel,Café,Coffee Shop,Italian Restaurant,Gastropub,Pizza Place,Restaurant,Chinese Restaurant,Museum,Salad Place
1253,2217000.0,French Restaurant,Pub,Brewery,Bakery,Coffee Shop,Lake,Train Station,Chinese Restaurant,Farmers Market,Event Space
2225,2200000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2637,2250000.0,Construction & Landscaping,Gastropub,Food Stand,Fast Food Restaurant,Ethiopian Restaurant,Event Space,Exhibit,Factory,Falafel Restaurant,Farm


In [64]:
london_grouped_clustering.loc[london_grouped_clustering['Cluster Labels'] == 1, london_grouped_clustering.columns[[1] + list(range(5, london_grouped_clustering.shape[1]))]].head()

,Avg_Price,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
196,2450000.0,Café,Bar,Indian Restaurant,Restaurant,Pub,Coffee Shop,Museum,Burger Joint,Supermarket,Seafood Restaurant
981,2480000.0,Speakeasy,Spa,Pizza Place,Automotive Shop,Filipino Restaurant,Event Space,Exhibit,Factory,Falafel Restaurant,Farm
1553,2456875.0,Home Service,Zoo,Filipino Restaurant,Ethiopian Restaurant,Event Space,Exhibit,Factory,Falafel Restaurant,Farm,Farmers Market
1980,2492500.0,Supermarket,English Restaurant,Pub,American Restaurant,Gym,Coffee Shop,Rental Car Location,Fast Food Restaurant,Park,Café
2136,2461000.0,Soccer Field,Spa,Sports Club,Windmill,Bus Station,Fast Food Restaurant,Event Space,Exhibit,Factory,Falafel Restaurant


In [65]:
london_grouped_clustering.loc[london_grouped_clustering['Cluster Labels'] == 2, london_grouped_clustering.columns[[1] + list(range(5, london_grouped_clustering.shape[1]))]].head()

,Avg_Price,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
855,2375000.0,Indie Movie Theater,Thai Restaurant,Movie Theater,Food & Drink Shop,Nature Preserve,Farmers Market,Park,Pizza Place,Pub,Coffee Shop
1102,2340000.0,Pizza Place,Discount Store,Outlet Store,American Restaurant,Salon / Barbershop,Italian Restaurant,Chinese Restaurant,Zoo,Farm,Event Space
2068,2375000.0,Pub,Park,Hotel,Grocery Store,Pizza Place,Ice Cream Shop,Indian Restaurant,Yoga Studio,Italian Restaurant,Farmers Market
2129,2379652.7,Pub,Grocery Store,Indian Restaurant,Hotel,Pizza Place,Italian Restaurant,Coffee Shop,Bakery,Ice Cream Shop,Yoga Studio
2943,2367500.0,Hotel,Pub,Garden,Coffee Shop,Café,Italian Restaurant,Chinese Restaurant,Mediterranean Restaurant,Bar,Bakery


In [66]:
london_grouped_clustering.loc[london_grouped_clustering['Cluster Labels'] == 3, london_grouped_clustering.columns[[1] + list(range(5, london_grouped_clustering.shape[1]))]].head()

,Avg_Price,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
390,2435000.0,Pub,Grocery Store,Japanese Restaurant,English Restaurant,Gym / Fitness Center,Diner,Garden,French Restaurant,Park,Plaza
422,2400000.0,Pub,Casino,Bar,Nightclub,Fast Food Restaurant,Event Space,Exhibit,Factory,Falafel Restaurant,Farm
1632,2397132.0,Convenience Store,Coffee Shop,Sushi Restaurant,Grocery Store,Gym / Fitness Center,Costume Shop,Convention Center,Event Space,Exhibit,Factory
1797,2400000.0,Art Gallery,Zoo,Fish & Chips Shop,Event Space,Exhibit,Factory,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant
1914,2445000.0,Athletics & Sports,Italian Restaurant,Bakery,Greek Restaurant,Stadium,Art Gallery,Grocery Store,Pizza Place,Food & Drink Shop,Pub


In [67]:
london_grouped_clustering.loc[london_grouped_clustering['Cluster Labels'] == 4, london_grouped_clustering.columns[[1] + list(range(5, london_grouped_clustering.shape[1]))]].head()

,Avg_Price,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2242,2.300000e+06,Farm,Soup Place,Zoo,Filipino Restaurant,Ethiopian Restaurant,Event Space,Exhibit,Factory,Falafel Restaurant,Farmers Market
2405,2.286679e+06,Café,Pub,Bar,Coffee Shop,Italian Restaurant,Park,French Restaurant,Restaurant,Gourmet Shop,Beer Store
2685,2.287500e+06,Pub,Brewery,Gift Shop,Art Museum,Gym / Fitness Center,Farmers Market,Ethiopian Restaurant,Event Space,Exhibit,Factory
3376,2.298000e+06,Hotel,Zoo,Filipino Restaurant,Ethiopian Restaurant,Event Space,Exhibit,Factory,Falafel Restaurant,Farm,Farmers Market
4284,2.265000e+06,Pub,Electronics Store,Ethiopian Restaurant,Event Space,Exhibit,Factory,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant


<h2>Results and Discussion section</h2>

<p>First of all, even though the London Housing Market may be in a rut, it is still an "ever-green" for business affairs.</p>
<p>We may discuss our results under two main perspectives.</p>
<p>First, we may examine them according to neighborhoods/London areas. It is interesting to note that, although West London (Notting Hill, Kensington, Chelsea, Marylebone) and North-West London (Hampsted) might be considered highly profitable venues to purchase a real estate according to amenities and essential facilities surrounding such venues i.e. elementary schools, high schools, hospitals & grocery stores, South-West London (Wandsworth, Balham) and North-West London (Isliington) are arising as next future elite venues with a wide range of amenities and facilities. Accordingly, one might target under-priced real estates in these areas of London in order to make a business affair.</p>
<p>First, we may examine them according to neighborhoods/London areas. It is interesting to note that, although West London (Notting Hill, Kensington, Chelsea, Marylebone) and North-West London (Hampsted) might be considered highly profitable venues to purchase a real estate according to amenities and essential facilities surrounding such venues i.e. elementary schools, high schools, hospitals & grocery stores, South-West London (Wandsworth, Balham) and North-West London (Isliington) are arising as next future elite venues with a wide range of amenities and facilities. Accordingly, one might target under-priced real estates in these areas of London in order to make a business affair.</p>


<h2>Conclusion</h2>
<p>To sum up, according to Bloomberg News, the London Housing Market is in a rut. It is now facing a number of different headwinds, including the prospect of higher taxes and a warning from the Bank of England that U.K. home values could fall as much as 30 percent in the event of a disorderly exit from the European Union. In this scenario, it is urgent to adopt machine learning tools in order to assist homebuyers clientele in London to make wise and effective decisions. As a result, the business problem we were posing was: how could we provide support to homebuyers clientele in to purchase a suitable real estate in London in this uncertain economic and financial scenario?</p>
<p>To solve this business problem, we clustered London neighborhoods in order to recommend venues and the current average price of real estate where homebuyers can make a real estate investment. We recommended profitable venues according to amenities and essential facilities surrounding such venues i.e. elementary schools, high schools, hospitals & grocery stores.</p>
<p>First, we gathered data on London properties and the relative price paid data were extracted from the HM Land Registry (http://landregistry.data.gov.uk/). Moreover, to explore and target recommended locations across different venues according to the presence of amenities and essential facilities, we accessed data through FourSquare API interface and arranged them as a data frame for visualization. By merging data on London properties and the relative price paid data from the HM Land Registry and data on amenities and essential facilities surrounding such properties from FourSquare API interface, we were able to recommend profitable real estate investments.</p>
<p>Second, The Methodology section comprised four stages: 1. Collect Inspection Data; 2. Explore and Understand Data; 3. Data preparation and preprocessing; 4. Modeling. In particular, in the modeling section, we used the k-means clustering technique as it is fast and efficient in terms of computational cost, is highly flexible to account for mutations in real estate market in London and is accurate.</p>
<p>Finally, we drew the conclusion that even though the London Housing Market may be in a rut, it is still an "ever-green" for business affairs. We discussed our results under two main perspectives. First, we examined them according to neighborhoods/London areas. although West London (Notting Hill, Kensington, Chelsea, Marylebone) and North-West London (Hampsted) might be considered highly profitable venues to purchase a real estate according to amenities and essential facilities surrounding such venues i.e. elementary schools, high schools, hospitals & grocery stores, South-West London (Wandsworth, Balham) and North-West London (Isliington) are arising as next future elite venues with a wide range of amenities and facilities. Accordingly, one might target under-priced real estates in these areas of London in order to make a business affair. Second, we analyzed our results according to the five clusters we produced. While Clusters 0, 2 and 4 may target home buyers prone to live in 'green' areas with parks, waterfronts, Clusters 1 and 3 may target individuals who love pubs, theatres and soccer.</p>